# Imports

## Libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta

## Data Ingestion

In [ ]:
morning_path = '/content/morning_routine.xlsx'
night_path = '/content/night_routine.xlsx'

In [ ]:
df_m = pd.read_excel(morning_path)
df_n = pd.read_excel(night_path)

# Functions

### [*Function*] Field Type Formatting


In [ ]:
# Function to apply specific rules to timestamp columns in a DataFrame
def timestamp_rules(dataframe, column):
    # Converts column values to datetime.time objects
    dataframe[column] = pd.to_datetime(dataframe[column], format='%H:%M:%S').dt.time

    # Checks if the column is hr_fill or hr_wakeup
    if column in ("hr_fill", "hr_wakeup"):
        # Retains the time of the hr_fill or hr_wakeup field and uses the day based the dt_sleep field
        dataframe[column] = dataframe.apply(lambda row: datetime.combine(row["dt_sleep"], row[column]), axis=1)
    # If the column is hr_asleep
    elif column == "hr_asleep":
        # Retains the time of the hr_asleep field while decreasing the date by one day based on the dt_sleep field
        dataframe[column] = dataframe.apply(lambda row: datetime.combine(row["dt_sleep"] + timedelta(days=-1), row[column]), axis=1)

    # Returns the modified DataFrame
    return dataframe

# Main function to convert data types in a DataFrame based on a type mapping
def convert_dtype(dataframe, map_dtypes):
    # Iterates over the data type mapping
    for column, dtype in map_dtypes.items():
        # Checks the data type and applies the appropriate transformation
        if dtype == "time":
            # Calls the timestamp_rules function to handle date-with-time fields
            dataframe = timestamp_rules(dataframe, column)
        elif dtype == "date":
            # Converts the column to datetime.date objects
            dataframe[column] = pd.to_datetime(dataframe[column]).dt.date
        elif dtype == "duration":
            # Converts duration fields to timedelta
            dataframe[column] = pd.to_datetime(dataframe[column], format='%H:%M:%S').dt.time.apply(lambda x: timedelta(hours=x.hour, minutes=x.minute, seconds=x.second))

    # Returns the modified DataFrame
    return dataframe

# Data Modeling

## Sleep Data

In [ ]:
sleep_columns = [
    "Sobre qual data se trata os dados inseridos?",
    "Qual o horário de preenchimento desse formulário?",
    "Qual o horário adormecido ontem?",
    "Qual o horário levantado hoje?",
    "Qual a duração do sono de ontem para hoje?",
    "Qual o tempo de sono profundo durante a noite?",
    "Qual o tempo de sono leve durante a noite?",
    "Qual o tempo acordado durante a noite?"
]

sleep_rename_dict = {
    "Sobre qual data se trata os dados inseridos?": "dt_sleep",
    "Qual o horário de preenchimento desse formulário?": "hr_fill",
    "Qual o horário adormecido ontem?": "hr_asleep",
    "Qual o horário levantado hoje?": "hr_wakeup",
    "Qual a duração do sono de ontem para hoje?": "ln_full_sleep",
    "Qual o tempo de sono profundo durante a noite?": "ln_deep_sleep",
    "Qual o tempo de sono leve durante a noite?": "ln_light_sleep",
    "Qual o tempo acordado durante a noite?": "ln_wakeup"
}

sleep_map_dtypes = {
    "dt_sleep": "date",
    "hr_fill": "time",
    "hr_asleep": "time",
    "hr_wakeup": "time",
    "ln_full_sleep": "duration",
    "ln_deep_sleep": "duration",
    "ln_light_sleep": "duration",
    "ln_wakeup": "duration"
}

# Aplicar a transformação
df_sleep = convert_dtype((df_m[sleep_columns].rename(columns=sleep_rename_dict)), sleep_map_dtypes)



In [ ]:
df_sleep.head(10)

,dt_sleep,hr_fill,hr_asleep,hr_wakeup,ln_full_sleep,ln_deep_sleep,ln_light_sleep,ln_wakeup
0,2023-11-08,2023-11-08 07:42:00,2023-11-07 20:58:00,2023-11-08 04:50:00,0 days 07:52:00,0 days 01:49:00,0 days 05:58:00,0 days 00:05:00
1,2023-11-07,2023-11-07 06:58:00,2023-11-06 20:55:00,2023-11-07 04:50:00,0 days 07:55:00,0 days 02:46:00,0 days 05:09:00,0 days 00:00:00
2,2023-11-09,2023-11-09 05:21:00,2023-11-08 21:39:00,2023-11-09 05:05:00,0 days 07:26:00,0 days 02:10:00,0 days 05:11:00,0 days 00:05:00
3,2023-11-10,2023-11-10 05:10:00,2023-11-09 20:47:00,2023-11-10 05:03:00,0 days 08:16:00,0 days 02:41:00,0 days 05:30:00,0 days 00:05:00
4,2023-11-11,2023-11-11 05:01:00,2023-11-10 21:37:00,2023-11-11 04:52:00,0 days 07:15:00,0 days 02:47:00,0 days 04:26:00,0 days 00:02:00
5,2023-11-12,2023-11-12 09:59:00,2023-11-11 22:49:00,2023-11-12 08:07:00,0 days 09:18:00,0 days 01:54:00,0 days 07:16:00,0 days 00:08:00
6,2023-11-13,2023-11-13 04:57:00,2023-11-12 21:26:00,2023-11-13 04:20:00,0 days 06:54:00,0 days 01:54:00,0 days 04:56:00,0 days 00:04:00
7,2023-11-14,2023-11-14 05:19:00,2023-11-13 20:57:00,2023-11-14 05:10:00,0 days 08:13:00,0 days 02:55:00,0 days 05:13:00,0 days 00:05:00
8,2023-11-15,2023-11-15 05:30:00,2023-11-14 22:43:00,2023-11-15 05:18:00,0 days 06:35:00,0 days 01:59:00,0 days 04:26:00,0 days 00:10:00
9,2023-11-16,2023-11-16 07:38:00,2023-11-15 00:07:00,2023-11-16 07:30:00,0 days 07:23:00,0 days 02:33:00,0 days 04:36:00,0 days 00:14:00


## Weighing Data

In [ ]:
colunas_sobrando = [x for x in df_m.columns if x not in sleep_columns]
colunas_sobrando

['Carimbo de data/hora',
 'Endereço de e-mail',
 'Como está se iniciando o dia?',
 'Qual o tempo de uso de celular ontem?',
 'Qual o peso total de hoje? [X_.__]',
 'Qual o peso total de hoje? [_X.__]',
 'Qual o peso total de hoje? [__.X_]',
 'Qual o peso total de hoje? [__._X]',
 'Qual a massa magra de hoje? [X_.__]',
 'Qual a massa magra de hoje? [_X.__]',
 'Qual a massa magra de hoje? [__.X_]',
 'Qual a massa magra de hoje? [__._X]',
 'Qual o percentual de gordura (bf) de hoje? [X_.__]',
 'Qual o percentual de gordura (bf) de hoje? [_X.__]',
 'Qual o percentual de gordura (bf) de hoje? [__.X_]',
 'Qual o percentual de gordura (bf) de hoje? [__._X]',
 'Qual metabolismo basal aproximado?',
 'Quantas calorias foram queimadas no dia anterior?',
 'Qual é a priorização ideal para o dia de hoje? [Sabedoria]',
 'Qual é a priorização ideal para o dia de hoje? [Estabilidade]',
 'Qual é a priorização ideal para o dia de hoje? [Força]',
 'Qual é a priorização ideal para o dia de hoje? [Gentileza

## Exercises Data